# Imports

In [12]:
from vpython import *
import ikpy.chain
import ikpy.utils.plot as plot_utils
import xml.etree.ElementTree as ET
import numpy as np
import math
my_chain = ikpy.chain.Chain.from_urdf_file("arm_urdf.urdf",active_links_mask=[False, True, True, True, True, True])

# definitions

In [9]:
def axisAngleRotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
class Link:
    def __init__(self, pos=np.zeros(3), size=np.ones(3), axis=None, jointOrigin=None):
        self.pos = pos
        temp = self.pos[2]
        self.pos[2] = self.pos[1]
        self.pos[1] = temp
        self.size = size
        temp = self.size[2]
        self.size[2] = self.size[1]
        self.size[1] = temp
        if np.any(axis == None):
            self.jointAxis = np.array([0., 1., 0.])
        else:
            self.jointAxis = axis
            temp = self.jointAxis[2]
            self.jointAxis[2] = self.jointAxis[1]
            self.jointAxis[1] = temp
        if np.any(jointOrigin == None):
            self.jointOrigin = self.pos.copy()
        else:
            self.jointOrigin = jointOrigin
            temp = self.jointOrigin[2]
            self.jointOrigin[2] = self.jointOrigin[1]
            self.jointOrigin[1] = temp
        self.angle = 0
        self.axis = np.array([1., 0., 0.])
        self.up = np.array([0., 1., 0.])
        self.visual = box(pos=vec(pos[0], pos[1], pos[2]), size=vec(size[0], size[1], size[2]), color=vec(0.5, 0.5, 1))
    def update(self, pos, angle):
        self.pos = pos
        self.visual.pos = vec(self.pos[0], self.pos[1], self.pos[2])
        self.axis = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.axis)
        self.up = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.up)
        self.visual.up = vec(*self.up)
        self.visual.axis = vec(*self.axis)
        self.visual.size = vec(*self.size)
        self.angle = angle
        
class Robot:
    def __init__(self, links):
        self.links = links
    def setAngle(self, linkIdx, angle):
        if linkIdx == 0:
            print("BASE LINK CANT MOVE")
            return
        # link = self.links[linkIdx]
        # dAngle = angle - link.angle
        # link.angle = angle
        # rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
        # link.axis = np.dot(rotMat, link.axis)
        # prevEndPoint = link.jointOrigin + np.max(link.size)*link.up
        # link.up = np.dot(rotMat, link.up)
        # endPoint = link.jointOrigin + np.max(link.size)*link.up
        # self.links[linkIdx+1].jointOrigin += (endPoint-prevEndPoint)
        # link.pos = link.jointOrigin + np.max(link.size)*link.up*0.5
        # link.visual.pos = vec(*link.pos)
        # link.visual.axis = vec(*link.axis)
        # link.visual.up = vec(*link.up)
        # link.visual.size = vec(*link.size)

        for i in range(linkIdx, len(self.links)):
            
            link = self.links[i]
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            if i == linkIdx:
                dAngle = angle - link.angle
                link.angle = angle
                rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
                prevEndPoint = link.jointOrigin + np.max(link.size)*moveAxis
            link.jointAxis = np.dot(rotMat, link.jointAxis)
            link.axis = np.dot(rotMat, link.axis)
            link.up = np.dot(rotMat, link.up)
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            endPoint = link.jointOrigin + np.max(link.size)*moveAxis
            if i < len(self.links)-1:
                temp = self.links[i+1].jointOrigin + np.max(self.links[i+1].size)*self.links[i+1].up
                # self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                # EXTREMELY HARD CODED DONT DO THIS LOL
                
                if i == 2:
                    self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                elif i == 3:
                    self.links[i+1].jointOrigin = endPoint+0.07*link.axis
                elif i == 1:
                    self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                elif i == 4:
                    self.links[i+1].jointOrigin = endPoint-0.01*link.up
                # elif i == 4:
                #     self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                prevEndPoint = temp
            if i == 5:
                link.pos = link.jointOrigin
            else:
                link.pos = link.jointOrigin + np.max(link.size)*moveAxis*0.5
            link.visual.pos = vec(*link.pos)
            link.visual.axis = vec(*link.axis)
            link.visual.up = vec(*link.up)
            link.visual.size = vec(*link.size)

        

# Run the sim

In [13]:
scene = canvas()


# Parse the URDF file
tree = ET.parse('arm_urdf.urdf')
root = tree.getroot()

# Access robot properties
robot_name = root.attrib.get('name')
# print(robot_name)
summedPos = np.zeros(3)
joints = {}
for joint in root.findall('joint'):
    parent = joint.findall('parent')[0].attrib['link']
    child = joint.findall('child')[0].attrib['link']
    origin = np.array([float(i) for i in joint.findall('origin')[0].attrib['xyz'].split(" ")])
    axis = np.array([float(i) for i in joint.findall('axis')[0].attrib['xyz'].split(" ")])
    # print("JOINT", parent, child, origin, axis)
    joints[child] = [origin, axis]

links = []
for link in root.findall('link'):
    name = link.attrib['name']
    # print(name)
    visual = link.findall('visual')[0]
    origin = visual.findall('origin')[0]
    geom = visual.findall('geometry')[0]
    if len(geom.findall('box'))>0:
        # print("BOX")
        geom = geom.findall('box')[0]
        size = np.array([float(i) for i in geom.attrib['size'].split(" ")])
    else:
        # print("CYLINDER")
        geom = geom.findall('cylinder')[0]
        radius = float(geom.attrib['radius'])
        length = float(geom.attrib['length'])
        size = np.array([radius*2, radius*2, length])
    origin = np.array([float(i) for i in origin.attrib['xyz'].split(" ")])
    
    axis = None
    jointOrigin = None
    if name in joints:
        jointOrigin = joints[name][0]
        axis = joints[name][1]
        summedPos += jointOrigin
    origin += summedPos
    
    # print("Pos", origin)
    # print("Size", size)
    links.append(Link(origin, size, axis, summedPos.copy()))
# links[3].visual.color=vec(1, 0, 0)
robot = Robot(links)
# robot.setAngle(1, 3.14159/4)
# robot.setAngle(2, 3.14159/4)
# robot.setAngle(3, 3.14159/4)
# robot.setAngle(4, 3.14159/4)
# robot.setAngle(5, 3.14159/4)

dt = 0.01
time = np.arange(0, 0.05, dt)
xposes = np.linspace(-.7, .7, len(time))
yposes = np.linspace(-.7, .7, len(time))
zposes = np.linspace(-.7, .7, len(time))

for t in time:

    target_position = [ 0.3048, 0.3048,0.1]


    ik = my_chain.inverse_kinematics(target_position)
    print("The degrees of each joints are : ", list(map(lambda r:math.degrees(r),ik.tolist())))
    print("The radians of each joints are : ", list(map(lambda r:r,ik.tolist())))
    computed_position = my_chain.forward_kinematics(ik)
    print("Computed position: %s, original position : %s" % (computed_position[:3, 3], target_position))
    print("Computed position (readable) : %s" % [ '%.2f' % elem for elem in computed_position[:3, 3] ])
    angles = ik.tolist()
    print(angles)
    for i in range(1, len(angles)):
        robot.setAngle(i, angles[i])

    
    # robot.setAngle(1, sin(t)*2*3.14159)
    # robot.setAngle(2, sin(t)*0.25*3.14159)
    # robot.setAngle(3, sin(t)*0.25*3.14159)
    # robot.setAngle(4, sin(t)*0.25*3.14159)
    # robot.setAngle(5, sin(t)*0.25*3.14159)
    t+= dt
    rate(100)


<IPython.core.display.Javascript object>

The degrees of each joints are :  [0.0, -1.5425562615489308e-09, -4.536944284972458e-09, -4.811910639877521e-09, -3.2393680742013786e-09, 5.729577951308233e-09]
The radians of each joints are :  [0.0, -2.692268566128365e-11, -7.918461575230929e-11, -8.39836839776099e-11, -5.6537638578802036e-11, 1e-10]
Computed position: [1.00000000e-01 1.16689332e-10 8.30000000e-01], original position : [0.3048, 0.3048, 0.1]
Computed position (readable) : ['0.10', '0.00', '0.83']
[0.0, -2.692268566128365e-11, -7.918461575230929e-11, -8.39836839776099e-11, -5.6537638578802036e-11, 1e-10]
The degrees of each joints are :  [0.0, -1.5425562615489308e-09, -4.536944284972458e-09, -4.811910639877521e-09, -3.2393680742013786e-09, 5.729577951308233e-09]
The radians of each joints are :  [0.0, -2.692268566128365e-11, -7.918461575230929e-11, -8.39836839776099e-11, -5.6537638578802036e-11, 1e-10]
Computed position: [1.00000000e-01 1.16689332e-10 8.30000000e-01], original position : [0.3048, 0.3048, 0.1]
Computed 